# Testing

## Imports

In [223]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Functions

In [224]:
def generate_random_rank_r_matrix(m):
    n = int(0.5 * m)
    r = int(np.floor(0.25 * m))
    while True:
        A = np.random.rand(m, n)
        U, S, VT = np.linalg.svd(A)
        S_bar = np.zeros((m, n))
        if S.shape[0] < r:
            continue
        for i in range(r):
            S_bar[i, i] = S[i]
        A = np.dot(U, np.dot(S_bar, VT))
        return A

In [225]:
def matrix_rank(H):
    U, S, VT = np.linalg.svd(H)
    rank = 0
    for i in range(S.shape[0]):
        if S[i] > epsilon:
            rank += 1
    return rank

In [226]:
def matrix_vec_inf_norm(H):
    return max([np.abs(h) for h in H.flatten()])

In [227]:
def matrix_frobenius_norm(H):
    return np.linalg.norm(H, ord="fro")

In [228]:
def initial_variables(V1, U1, D_inv, rho):
    Theta = np.dot(V1, U1.T) / matrix_vec_inf_norm(np.dot(V1, U1.T))
    Lambda = Theta / rho
    E = np.dot(V1, np.dot(D_inv, U1.T)) + Lambda
    return Lambda, E

In [229]:
def soft_thresholding(a, kappa):
    if a > kappa:
        return a - kappa
    elif np.abs(a) <= kappa:
        return 0
    elif a < -kappa:
        return a + kappa

In [230]:
def admm1_123(A, rho, epsilon_abs, epsilon_rel):
    # Gets dimensions of matrix A
    m = A.shape[0]
    n = A.shape[1]
    # Calculates full singular value decomposition of A
    U, S, VT = np.linalg.svd(A)
    # Calculates rank of A
    r = matrix_rank(A)
    # Calculates variables U1, V1, V2 and D^{-1}
    U1 = U[:, :r]
    V1 = VT.T[:, :r]
    V2 = VT.T[:, r:]
    D_inv = np.zeros((r, r))
    for i in range(r):
            D_inv[i, i] = 1 / S[i]
    # Calculates initial variables
    Lambda, Ekm = initial_variables(V1=V1, U1=U1, D_inv=D_inv, rho=rho)
    Ek = np.zeros((n, m))
    while True:
        # Updates variable J
        J = Ekm - np.dot(V1, np.dot(D_inv, U1.T)) - Lambda
        # Updates variable Zk
        Z = np.dot(V2.T, np.dot(J, U1))
        # Updates variable Y
        Y = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T)) + Lambda
        # Updates variable Ek
        for i in range(n):
            for j in range(m):
                Ek[i, j] = soft_thresholding(a=Y[i, j], kappa=1/rho)
        # Updates variable Lambdak
        Lambda = Lambda + np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T)) - Ek
        # Calculates stop criterion variables
        rk = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T)) - Ek
        sk = rho * np.dot(V2.T, np.dot(Ek - Ekm, U1))
        matrix_norms = [
            matrix_frobenius_norm(Ek),
            matrix_frobenius_norm(np.dot(V2, np.dot(Z, U1.T))),
            matrix_frobenius_norm(np.dot(V1, np.dot(D_inv, U1.T)))
        ]
        primal_upper_bound = epsilon_abs * np.sqrt(m*n) + epsilon_rel * max(matrix_norms)
        aux_var = matrix_frobenius_norm(np.dot(V2.T, np.dot(Lambda, U1)))
        dual_upper_bound = epsilon_abs * np.sqrt((n-r)*r) + epsilon_rel * rho * aux_var
        # Checks stop criterion
        if (matrix_frobenius_norm(rk) <= primal_upper_bound) and (matrix_frobenius_norm(sk)) < dual_upper_bound:
            break
        # Makes Ek the new Ek-1
        Ekm = Ek
    # Calculates output matrix H
    H = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T))
    return H

In [231]:
def admm1_134(A, rho, epsilon_abs, epsilon_rel):
    # Gets dimensions of matrix A
    m = A.shape[0]
    n = A.shape[1]
    # Calculates full singular value decomposition of A
    U, S, VT = np.linalg.svd(A)
    # Calculates rank of A
    r = matrix_rank(A)
    # Calculates variables U1, U2, V1, V2 and D^{-1}
    U1 = U[:, :r]
    U2 = U[:, r:]
    V1 = VT.T[:, :r]
    V2 = VT.T[:, r:]
    D_inv = np.zeros((r, r))
    for i in range(r):
            D_inv[i, i] = 1 / S[i]
    # Calculates initial variables
    Lambda, Ekm = initial_variables(V1=V1, U1=U1, D_inv=D_inv, rho=rho)
    Ek = np.zeros((n, m))
    while True:
        # Updates variable J
        J = Ekm - np.dot(V1, np.dot(D_inv, U1.T)) - Lambda
        # Updates variable Wk
        W = np.dot(V2.T, np.dot(J, U2))
        # Updates variable Y
        Y = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T)) + Lambda
        # Updates variable Ek
        for i in range(n):
            for j in range(m):
                Ek[i, j] = soft_thresholding(a=Y[i, j], kappa=1/rho)
        # Updates variable Lambdak
        Lambda = Lambda + np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T)) - Ek
        # Calculates stop criterion variables
        rk = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T)) - Ek
        sk = rho * np.dot(V2.T, np.dot(Ek - Ekm, U1))
        matrix_norms = [
            matrix_frobenius_norm(Ek),
            matrix_frobenius_norm(np.dot(V2, np.dot(W, U2.T))),
            matrix_frobenius_norm(np.dot(V1, np.dot(D_inv, U1.T)))
        ]
        primal_upper_bound = epsilon_abs * np.sqrt(m*n) + epsilon_rel * max(matrix_norms)
        aux_var = matrix_frobenius_norm(np.dot(V2.T, np.dot(Lambda, U1)))
        dual_upper_bound = epsilon_abs * np.sqrt((n-r)*r) + epsilon_rel * rho * aux_var
        # Checks stop criterion
        if (matrix_frobenius_norm(rk) <= primal_upper_bound) and (matrix_frobenius_norm(sk)) < dual_upper_bound:
            break
        # Makes Ek the new Ek-1
        Ekm = Ek
    # Calculates output matrix H
    H = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T))
    return H

## Testing Functions

In [232]:
epsilon = 10 ** -5

m = 10
A = generate_random_rank_r_matrix(m=m)
rho = 0.5
epsilon_abs = 10 ** -5
epsilon_rel = 10 ** -6

In [233]:
H = admm1_123(A=A, rho=rho, epsilon_abs=epsilon_abs, epsilon_rel=epsilon_rel)

AHA = np.dot(A, np.dot(H, A))
HAH = np.dot(H, np.dot(A, H))
AH = np.dot(A, H)
AH_T = np.dot(A, H).T
HA = np.dot(H, A)
HA_T = np.dot(H, A).T
print(matrix_frobenius_norm(AHA - A))
print(matrix_frobenius_norm(HAH - H))
print(matrix_frobenius_norm(AH - AH_T))
print(matrix_frobenius_norm(HA - HA_T))

1.3101756731727149e-15
2.5958252401743736e-16
4.647845098012396e-16
2.112030322619477


In [234]:
H = admm1_134(A=A, rho=rho, epsilon_abs=epsilon_abs, epsilon_rel=epsilon_rel)

AHA = np.dot(A, np.dot(H, A))
HAH = np.dot(H, np.dot(A, H))
AH = np.dot(A, H)
AH_T = np.dot(A, H).T
HA = np.dot(H, A)
HA_T = np.dot(H, A).T
print(matrix_frobenius_norm(AHA - A))
print(matrix_frobenius_norm(HAH - H))
print(matrix_frobenius_norm(AH - AH_T))
print(matrix_frobenius_norm(HA - HA_T))

1.4481034772981419e-15
0.7836509200128745
5.571474341854064e-16
4.769700983088376e-16


In [235]:
print(np.linalg.norm(H.flatten(), ord=0))
total = 0
for x in H.flatten():
    if np.abs(x) > 10 ** -5:
        total += 1
print(total)
print(H)

50.0
36
[[-6.15588801e-02  3.02918062e-05 -5.70488575e-02  4.24492608e-01
   2.03386365e-05  7.13942570e-02  4.36501301e-01 -7.68071204e-02
  -1.03531118e-01 -1.27601178e-01]
 [-5.94716508e-06  2.37505888e-01 -4.38425935e-02 -7.03501011e-06
   1.87898887e-06  7.22862618e-02 -7.38537541e-06 -1.19976314e-05
  -6.38929341e-06 -1.05844897e-05]
 [-2.24146362e-06 -7.56687166e-06  4.93451735e-01 -1.12257723e-06
   6.33307419e-02 -2.20352910e-01 -1.20656075e-06 -4.37620051e-06
  -2.52306637e-06  1.06159457e-01]
 [ 1.61286761e-01 -5.72831302e-06  5.31420015e-06 -1.81363787e-01
   1.50971763e-06 -1.57099692e-01 -1.83562259e-01  2.89919282e-01
   2.01261609e-01  2.16997191e-01]
 [-1.01672923e-05  1.05089562e-01 -3.07015829e-01 -1.41167155e-05
   1.65792527e-01  4.43006459e-01 -1.47813987e-05 -2.07102730e-05
  -1.07660240e-05 -1.95085108e-05]]
